## Classifying a restaurant review to be Good or Bad

In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [3]:
data['review'] = data['review'].apply(lambda s: str(s))

In [5]:
#1 for good
#0 for bad
def binarize(n):
    if n >= 3:
        return 1
    else:
        return 0

data['rating'] = data['rating'].apply(lambda s: binarize(s))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['rating'], test_size=0.30, random_state=42)

In [7]:
data.groupby('rating').count()

,index,user,item,review,real,date
rating,,,,,,
0,78577,78577,78577,78577,78577,78577
1,529881,529881,529881,529881,529881,529881


In [8]:
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(X_train)
train_sequence = pad_sequences(sequences, maxlen=100)

In [9]:
ip = Input(shape=[100])
lyr = Embedding(2000, 32, input_length=100)(ip)
lyr = LSTM(256, recurrent_dropout=0.2)(lyr)
lyr = Dense(256)(lyr)
lyr = Dropout(0.3)(lyr)
lyr = Dense(1)(lyr)
lyr = Activation('sigmoid')(lyr)
model = Model(inputs=ip, outputs=lyr)

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total para

In [10]:
model.fit(train_sequence, y_train, batch_size=1000, epochs=1, verbose = 1)

Epoch 1/1


204000/425920 [=============>................] - ETA: 35:59 - loss: 0.6927 - acc: 0.56 - ETA: 32:22 - loss: 0.6654 - acc: 0.72 - ETA: 31:22 - loss: 0.5823 - acc: 0.77 - ETA: 30:40 - loss: 0.5919 - acc: 0.80 - ETA: 30:01 - loss: 0.5893 - acc: 0.81 - ETA: 29:49 - loss: 0.5772 - acc: 0.82 - ETA: 29:32 - loss: 0.5534 - acc: 0.82 - ETA: 29:16 - loss: 0.5976 - acc: 0.83 - ETA: 28:54 - loss: 0.5884 - acc: 0.83 - ETA: 28:33 - loss: 0.5785 - acc: 0.84 - ETA: 28:42 - loss: 0.5679 - acc: 0.84 - ETA: 28:43 - loss: 0.5573 - acc: 0.84 - ETA: 28:51 - loss: 0.5465 - acc: 0.84 - ETA: 29:00 - loss: 0.5356 - acc: 0.85 - ETA: 29:03 - loss: 0.5237 - acc: 0.85 - ETA: 29:23 - loss: 0.5151 - acc: 0.85 - ETA: 29:43 - loss: 0.5049 - acc: 0.85 - ETA: 29:39 - loss: 0.4985 - acc: 0.85 - ETA: 29:32 - loss: 0.4918 - acc: 0.85 - ETA: 29:31 - loss: 0.4855 - acc: 0.85 - ETA: 29:23 - loss: 0.4803 - acc: 0.85 - ETA: 29:38 - loss: 0.4746 - acc: 0.85 - ETA: 29:41 - loss: 0.4702 - acc: 0.85 - ETA: 29:49 - loss: 0.4676 - acc

408000/425920 [===========================>..] - ETA: 14:57 - loss: 0.2996 - acc: 0.88 - ETA: 14:52 - loss: 0.2992 - acc: 0.88 - ETA: 14:48 - loss: 0.2989 - acc: 0.89 - ETA: 14:43 - loss: 0.2985 - acc: 0.89 - ETA: 14:38 - loss: 0.2985 - acc: 0.89 - ETA: 14:34 - loss: 0.2987 - acc: 0.89 - ETA: 14:30 - loss: 0.2986 - acc: 0.89 - ETA: 14:26 - loss: 0.2985 - acc: 0.89 - ETA: 14:22 - loss: 0.2984 - acc: 0.89 - ETA: 14:17 - loss: 0.2983 - acc: 0.89 - ETA: 14:13 - loss: 0.2982 - acc: 0.89 - ETA: 14:08 - loss: 0.2980 - acc: 0.89 - ETA: 14:04 - loss: 0.2978 - acc: 0.89 - ETA: 13:59 - loss: 0.2974 - acc: 0.89 - ETA: 13:55 - loss: 0.2972 - acc: 0.89 - ETA: 13:51 - loss: 0.2970 - acc: 0.89 - ETA: 13:46 - loss: 0.2966 - acc: 0.89 - ETA: 13:42 - loss: 0.2963 - acc: 0.89 - ETA: 13:37 - loss: 0.2960 - acc: 0.89 - ETA: 13:33 - loss: 0.2957 - acc: 0.89 - ETA: 13:28 - loss: 0.2953 - acc: 0.89 - ETA: 13:24 - loss: 0.2949 - acc: 0.89 - ETA: 13:19 - loss: 0.2944 - acc: 0.89 - ETA: 13:15 - loss: 0.2941 - acc

425920/425920 [==============================] - ETA: 1:02 - loss: 0.2644 - acc: 0.898 - ETA: 58s - loss: 0.2643 - acc: 0.898 - ETA: 55s - loss: 0.2642 - acc: 0.89 - ETA: 51s - loss: 0.2640 - acc: 0.89 - ETA: 47s - loss: 0.2639 - acc: 0.89 - ETA: 44s - loss: 0.2640 - acc: 0.89 - ETA: 40s - loss: 0.2641 - acc: 0.89 - ETA: 36s - loss: 0.2640 - acc: 0.89 - ETA: 32s - loss: 0.2639 - acc: 0.89 - ETA: 29s - loss: 0.2638 - acc: 0.89 - ETA: 25s - loss: 0.2637 - acc: 0.89 - ETA: 21s - loss: 0.2637 - acc: 0.89 - ETA: 18s - loss: 0.2635 - acc: 0.89 - ETA: 14s - loss: 0.2634 - acc: 0.89 - ETA: 10s - loss: 0.2632 - acc: 0.89 - ETA: 7s - loss: 0.2630 - acc: 0.8993 - ETA: 3s - loss: 0.2628 - acc: 0.899 - 1572s 4ms/step - loss: 0.2627 - acc: 0.8994


In [11]:
test_seq = tokenizer.texts_to_sequences(X_test)
test_sequence = pad_sequences(test_seq, maxlen=100)

In [12]:
y_pred = model.predict(test_sequence)

In [23]:
tst = []
for i in y_pred:
    if i >= 0.50:
        tst.append(1)
    else:
        tst.append(0)

from sklearn.metrics import classification_report
print(classification_report(y_test, tst))

             precision    recall  f1-score   support

          0       0.74      0.56      0.64     23594
          1       0.94      0.97      0.95    158944

avg / total       0.91      0.92      0.91    182538



In [14]:
results = model.evaluate(test_sequence, y_test)

 50048/182538 [=======>......................] - ETA: 18:5 - ETA: 9:1 - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: 

 77376/182538 [===========>..................] - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - E

105216/182538 [================>.............] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E

131744/182538 [====================>.........] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E

158624/182538 [=========================>....] - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - E

182538/182538 [==============================] - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - E

In [15]:
print("Accuracy:", results[1]*100, "%")

Accuracy: 91.77102849817572 %


In [16]:
#Saving the model
from keras.models import load_model
model.save(r'C:\Users\elonm\Desktop\MachineLearningProject\good_bad_model.h5')

In [ ]:
#Loading model
model = load_model('my_model.h5')

In [47]:
test_str = input("Enter a test string:")
test_str = test_str.lower()
tst_seq = tokenizer.texts_to_sequences([test_str])
tst_s = pad_sequences(tst_seq, maxlen=100)
tst_res = model.predict(tst_s)
if tst_res > 0.7:
    print("Good Review!")
else:
    print("Bad Review.")

Enter a test string:The food tasted bad
Bad Review.


In [48]:
print(tst_res[0][0])

0.63543844
